# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [1]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [2]:
import pandas as pd
import numpy as np
import re
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [3]:
# Path hack to allow imports from the parent directory.
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

In [4]:
from isadream.isadream import modelUtils
from isadream.isadream import io
from isadream.isadream.models import nodal, elemental

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [5]:
# A demo json file is provided.
nmr_json_demo = modelUtils.SIPOS_DEMO
demo_base_path = modelUtils.BASE_PATH
print(nmr_json_demo, '\n', demo_base_path)

/home/tyler/git/isadream/isadream/demo_data/demo_json/sipos_2006_talanta_nmr_figs.json 
 /home/tyler/git/isadream/isadream/demo_data


---

In [23]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

In [24]:
# node

In [25]:
node.assays[0].samples[0].get_param_values()

[('comments', []),
 ('factors',
  [Factor(csv_column_index=0, decimal_value=0.0, factor_type='Measurement Condition', name='Factor00154', reference_value='', string_value='', unit_reference='Molar')]),
 ('name', 'SampleNode00157'),
 ('sample_name', 'Potassium Hydroxide'),
 ('sources', []),
 ('species',
  [SpeciesFactor(name='SpeciesFactor00155', species_reference='K+', stoichiometry=1.0),
   SpeciesFactor(name='SpeciesFactor00156', species_reference='OH-', stoichiometry=1.0)])]

In [29]:
help(node.assays[0].samples[0])

Help on SampleNode in module isadream.isadream.models.nodal object:

class SampleNode(param.parameterized.Parameterized)
 |  params(comments=List, factors=List, sample_name=String, sources=List, species=List, name=String)
 |  Model for a physical of simulated sample.
 |  
 |      A Sample object is a collection of species and factors.
 |  
 |      
 |  Parameters of 'SampleNode'
 |  ==========================
 |  
 |  Parameters changed from their default values are marked in red.
 |  Soft bound values are marked in cyan.
 |  C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None
 |  
 |  Name         Value  Type     Bounds     Mode  
 |  
 |  comments      []   List   (0, None)  V RW AN 
 |  factors       []   List   (0, None)  V RW AN 
 |  sample_name   ''  String               V RW  
 |  sources       []   List   (0, None)  V RW AN 
 |  species       []   List   (0, None)    V RW  
 |  
 |  Parameter docstrings:
 |  =====================
 |  
 |  comments:    A list of co

In [30]:
# help(node)

In [35]:
demo_json = io.read_idream_json(nmr_json_demo)
node = io.parse_node_json(demo_json)

test_groups = (
    ('27 Al ppm', 'ppm', ("Al",)),
    ('Total Aluminate Concentration', 'Molar', ("Al", )),
    ('Counter Ion Concentration', 'Molar', ("Na+", "Li+", "Cs+", "K+", )),
    ('Counter Ion', 'Species', ("Na+", "Li+", "Cs+", "K+", )),
    ('Base Concentration', 'Molar', ("OH-", )),
)

node_assays = [io.build_node_data(a, test_groups) for a in node.assays]
node_assays
dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df

,parent_node,assay_node,27 Al ppm,sample_node,Total Aluminate Concentration,Counter Ion Concentration,Counter Ion,Base Concentration
0,98553ea7-6748-4c84-b6ad-b8e983dbadea,9f05190d-4bfd-4941-9fea-3e7a151fce28,79.90,ea783460-8431-45f3-95f6-4ace0e7e5456,2.93,2.93,[<SpeciesFactor SpeciesFactor00323>],2.93
1,98553ea7-6748-4c84-b6ad-b8e983dbadea,9f05190d-4bfd-4941-9fea-3e7a151fce28,79.84,ea783460-8431-45f3-95f6-4ace0e7e5456,4.92,4.92,[<SpeciesFactor SpeciesFactor00323>],4.92
2,98553ea7-6748-4c84-b6ad-b8e983dbadea,9f05190d-4bfd-4941-9fea-3e7a151fce28,79.72,ea783460-8431-45f3-95f6-4ace0e7e5456,6.85,6.85,[<SpeciesFactor SpeciesFactor00323>],6.85
3,98553ea7-6748-4c84-b6ad-b8e983dbadea,9f05190d-4bfd-4941-9fea-3e7a151fce28,79.66,ea783460-8431-45f3-95f6-4ace0e7e5456,9.13,9.13,[<SpeciesFactor SpeciesFactor00323>],9.13
4,98553ea7-6748-4c84-b6ad-b8e983dbadea,9f05190d-4bfd-4941-9fea-3e7a151fce28,79.66,ea783460-8431-45f3-95f6-4ace0e7e5456,10.71,10.71,[<SpeciesFactor SpeciesFactor00323>],10.71
0,7a15cdb1-a8ef-49cc-8980-49bff511aa87,357ea156-935c-4df0-b870-cd15292b731c,79.92,22ca8bdd-20fa-4a53-99be-7461cbe8c7e8,0.66,0.66,[<SpeciesFactor SpeciesFactor00328>],0.66
1,7a15cdb1-a8ef-49cc-8980-49bff511aa87,357ea156-935c-4df0-b870-cd15292b731c,79.84,22ca8bdd-20fa-4a53-99be-7461cbe8c7e8,1.10,1.10,[<SpeciesFactor SpeciesFactor00328>],1.10
2,7a15cdb1-a8ef-49cc-8980-49bff511aa87,357ea156-935c-4df0-b870-cd15292b731c,79.66,22ca8bdd-20fa-4a53-99be-7461cbe8c7e8,1.64,1.64,[<SpeciesFactor SpeciesFactor00328>],1.64
3,7a15cdb1-a8ef-49cc-8980-49bff511aa87,357ea156-935c-4df0-b870-cd15292b731c,79.54,22ca8bdd-20fa-4a53-99be-7461cbe8c7e8,2.14,2.14,[<SpeciesFactor SpeciesFactor00328>],2.14
4,7a15cdb1-a8ef-49cc-8980-49bff511aa87,357ea156-935c-4df0-b870-cd15292b731c,79.36,22ca8bdd-20fa-4a53-99be-7461cbe8c7e8,2.59,2.59,[<SpeciesFactor SpeciesFactor00328>],2.59


In [36]:
metadata = [md for _, md in node_assays]
metadata
md = collections.ChainMap(*metadata)
md

ChainMap({'98553ea7-6748-4c84-b6ad-b8e983dbadea': (None, [Comment(body='I manually pulled this points out with a web tool.', comment_name='', name='Study level comment on Sipos 2006.')]), '9f05190d-4bfd-4941-9fea-3e7a151fce28': ('Potassium Hydroxide Assay', [Comment(body='I manually pulled this points out with a web tool.', comment_name='', name='Data extraction method.')]), '132cba7d-9986-4003-bc13-a18cb8c6c19b': SampleNode(comments=[], factors=[Factor(csv_column_index=0, decimal_value=0.005, factor_type='Measurement Condition', name='Factor00315', reference_value='', string_value='', unit_reference='Molar')], name='SampleNode00320', sample_name='Aluminate Solution', sources=[SourceNode(comments=[], factors=[Factor(csv_column_index=0, decimal_value=0.98, factor_type='Material Property', name='Factor00317', reference_value='Purity by Weight', string_value='', unit_reference='Percent')], name='SourceNode00319', source_name='Aluminum Wire', species=[SpeciesFactor(name='SpeciesFactor00318

---